# Visualizing the History of Nobel Prize Winnners

The Nobel Prize has been among the most prestigious international awards since 1901. Each year, awards are bestowed in chemistry, literature, physics, physiology or medicine, economics, and peace. In addition to the honor, prestige, and substantial prize money, the recipient also gets a gold medal with an image of Alfred Nobel (1833 - 1896), who established the prize.

![](Nobel_Prize.png)

The Nobel Foundation has made a dataset available of all prize winners from the outset of the awards from 1901 to 2023. The dataset used in this project is from the Nobel Prize API and is available in the `nobel.csv` file in the `data` folder.

In this project, you'll get a chance to explore and answer several questions related to this prizewinning data. And we encourage you then to explore further questions that you're interested in!

Analyze Nobel Prize winner data and identify patterns by answering the following questions:
1. What is the most commonly awarded gender and birth country?
    - Store your answers as string variables `top_gender` and `top_country`. 

1. Which decade had the highest ratio of US-born Nobel Prize winners to total winners in all categories?
    - Store this as an integer called `max_decade_usa`. 

1. Which decade and Nobel Prize category combination had the highest proportion of female laureates?
    - Store this as a dictionary called `max_female_dict` where the decade is the key and the category is the value. There should only be one key:value pair. 

1. Who was the first woman to receive a Nobel Prize, and in what category?
    - Save your string answers as `first_woman_name` and `first_woman_category`. 

1. Which individuals or organizations have won more than one Nobel Prize throughout the years?
    - Store the full names in a list named `repeat_list`.

In [39]:
# Loading in required libraries
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt

In [40]:
data_path = "data/nobel.csv"
nobel_data = pd.read_csv(data_path)
nobel_data.head()

,year,category,prize,motivation,prize_share,laureate_id,laureate_type,full_name,birth_date,birth_city,birth_country,sex,organization_name,organization_city,organization_country,death_date,death_city,death_country
0,1901,Chemistry,The Nobel Prize in Chemistry 1901,"""in recognition of the extraordinary services ...",1/1,160,Individual,Jacobus Henricus van 't Hoff,1852-08-30,Rotterdam,Netherlands,Male,Berlin University,Berlin,Germany,1911-03-01,Berlin,Germany
1,1901,Literature,The Nobel Prize in Literature 1901,"""in special recognition of his poetic composit...",1/1,569,Individual,Sully Prudhomme,1839-03-16,Paris,France,Male,NaN,NaN,NaN,1907-09-07,Châtenay,France
2,1901,Medicine,The Nobel Prize in Physiology or Medicine 1901,"""for his work on serum therapy, especially its...",1/1,293,Individual,Emil Adolf von Behring,1854-03-15,Hansdorf (Lawice),Prussia (Poland),Male,Marburg University,Marburg,Germany,1917-03-31,Marburg,Germany
3,1901,Peace,The Nobel Peace Prize 1901,NaN,1/2,462,Individual,Jean Henry Dunant,1828-05-08,Geneva,Switzerland,Male,NaN,NaN,NaN,1910-10-30,Heiden,Switzerland
4,1901,Peace,The Nobel Peace Prize 1901,NaN,1/2,463,Individual,Frédéric Passy,1822-05-20,Paris,France,Male,NaN,NaN,NaN,1912-06-12,Paris,France


## What is the most commonly awarded gender and birth country?
### Store your answers as string variables `top_gender` and `top_country`. 

In [50]:
# Gender
top_gender = nobel_data.value_counts('sex').reset_index().iloc[0]['sex']
print(top_gender)

Male


In [51]:
# Country
top_country = nobel_data.value_counts('birth_country').reset_index().iloc[0]['birth_country']
print(top_country)

United States of America


## Which decade had the highest ratio of US-born Nobel Prize winners to total winners in all categories?
### Store this as an integer called `max_decade_usa`. 

In [43]:
# make decade column
nobel_data['decade'] = (np.floor(nobel_data['year']/10) * 10).astype(int)

# is_us_born column
nobel_data['is_us_born'] = nobel_data['birth_country'] == 'United States of America' 

max_decade_usa: int = 1900
max_decade_usa_winners = 1
max_decade_usa_winner_ratio: float = 1.0

for decade in range(1900, 2030, 10):
    winners_total = nobel_data.query(f"decade == {decade}")['year'].count()
    decade_winner = nobel_data.query(f"decade == {decade} and is_us_born == True")
    us_winners = decade_winner['is_us_born'].count()
    us_winner_ratio = winners_total / max_decade_usa_winners
    
    # Determine whether to replace the data with current decade data
    if us_winner_ratio > max_decade_usa_winner_ratio:
        max_decade_usa_winners = us_winners
        max_decade_usa = decade
        max_decade_usa_winner_ratio = us_winner_ratio
        

print("year:", max_decade_usa)
print("winners:", max_decade_usa_winners)
print("ratio:", max_decade_usa_winner_ratio)

year: 1950
winners: 21
ratio: 72.0


## Which decade and Nobel Prize category combination had the highest proportion of female laureates?
### Store this as a dictionary called `max_female_dict` where the decade is the key and the category is the value. There should only be one key:value pair. 

In [44]:
highest_prop_decade = 1900
highest_prop_ratio = 0
highest_prop_category:str = ''
for decade in range(1900, 2030, 10):
    decade_data = nobel_data.query(f"decade == {decade}") 
    
    # Error handling when no female winners
    if decade_data.query(f"sex == 'Female'")['category'].count() == 0:
        continue
    
    female_max_category:str = decade_data.query(f"sex == 'Female'")['category'].max()
    
    decade_category_total_winners = decade_data.query(f"category == '{female_max_category}'")['laureate_id'].count()
    decade_category_female_winners = decade_data.query(f"category == '{female_max_category}' and sex == 'Female'")['laureate_id'].count()
    decade_female_category_proportion =  decade_category_total_winners / decade_category_female_winners
    
    # Assign to global vars if this decade has higher proportion
    if decade_female_category_proportion > highest_prop_ratio:
        highest_prop_ratio = decade_female_category_proportion
        highest_prop_category = female_max_category
        highest_prop_decade = decade

max_female_dict = {highest_prop_decade: highest_prop_category}
print(max_female_dict)

{2010: 'Physics'}


## Who was the first woman to receive a Nobel Prize, and in what category?
### Save your string answers as `first_woman_name` and `first_woman_category`. 

In [45]:
female_nobel_data = nobel_data.query("sex == 'Female'").sort_values('year')
female_nobel_data.head()

,year,category,prize,motivation,prize_share,laureate_id,laureate_type,full_name,birth_date,birth_city,birth_country,sex,organization_name,organization_city,organization_country,death_date,death_city,death_country,decade,is_us_born
19,1903,Physics,The Nobel Prize in Physics 1903,"""in recognition of the extraordinary services ...",1/4,6,Individual,"Marie Curie, née Sklodowska",1867-11-07,Warsaw,Russian Empire (Poland),Female,NaN,NaN,NaN,1934-07-04,Sallanches,France,1900,False
29,1905,Peace,The Nobel Peace Prize 1905,NaN,1/1,468,Individual,"Baroness Bertha Sophie Felicita von Suttner, n...",1843-06-09,Prague,Austrian Empire (Czech Republic),Female,NaN,NaN,NaN,1914-06-21,Vienna,Austria,1900,False
51,1909,Literature,The Nobel Prize in Literature 1909,"""in appreciation of the lofty idealism, vivid ...",1/1,579,Individual,Selma Ottilia Lovisa Lagerlöf,1858-11-20,Mårbacka,Sweden,Female,NaN,NaN,NaN,1940-03-16,Mårbacka,Sweden,1900,False
62,1911,Chemistry,The Nobel Prize in Chemistry 1911,"""in recognition of her services to the advance...",1/1,6,Individual,"Marie Curie, née Sklodowska",1867-11-07,Warsaw,Russian Empire (Poland),Female,Sorbonne University,Paris,France,1934-07-04,Sallanches,France,1910,False
128,1926,Literature,The Nobel Prize in Literature 1926,"""for her idealistically inspired writings whic...",1/1,597,Individual,Grazia Deledda,1871-09-27,"Nuoro, Sardinia",Italy,Female,NaN,NaN,NaN,1936-08-15,Rome,Italy,1920,False


In [46]:
first_female_nobel = female_nobel_data.iloc[0]
first_woman_name = first_female_nobel['full_name']
first_woman_category = first_female_nobel['category']
print(f"{first_woman_name=}")
print(f"{first_woman_category=}")

first_woman_name='Marie Curie, née Sklodowska'
first_woman_category='Physics'


## Which individuals or organizations have won more than one Nobel Prize throughout the years?
### Store the full names in a list named `repeat_list`.

In [47]:
# Organizations
org_winner_count = nobel_data.value_counts('organization_name').rename('nobel_prize_count').reset_index()
org_mult_winner = org_winner_count.query('nobel_prize_count > 1')
print(org_mult_winner)

                               organization_name  nobel_prize_count
0                       University of California                 36
1                             Harvard University                 28
2    Massachusetts Institute of Technology (MIT)                 22
3                            Stanford University                 22
4                          University of Chicago                 19
..                                           ...                ...
97                      Johns Hopkins University                  2
98                          Amsterdam University                  2
99                 Université Libre de Bruxelles                  2
100                          École Polytechnique                  2
101               Wellcome Research Laboratories                  2

[102 rows x 2 columns]


In [48]:
# Individuals
individual_winner_count = nobel_data.value_counts(["full_name","birth_date"]).rename("nobel_prize_count").reset_index()
individual_mult_winner = individual_winner_count.query("nobel_prize_count > 1")
print(individual_mult_winner)


                     full_name  birth_date  nobel_prize_count
0             Frederick Sanger  1918-08-13                  2
1                 John Bardeen  1908-05-23                  2
2           Linus Carl Pauling  1901-02-28                  2
3  Marie Curie, née Sklodowska  1867-11-07                  2


In [49]:
individual_mult_winner_names = list(individual_mult_winner['full_name'])
organization_mult_winner_names = list(org_mult_winner['organization_name'])
repeat_list = individual_mult_winner_names + organization_mult_winner_names
print(repeat_list)

['Frederick Sanger', 'John Bardeen', 'Linus Carl Pauling', 'Marie Curie, née Sklodowska', 'University of California', 'Harvard University', 'Massachusetts Institute of Technology (MIT)', 'Stanford University', 'University of Chicago', 'California Institute of Technology (Caltech)', 'University of Cambridge', 'Columbia University', 'Princeton University', 'Rockefeller University', 'University of Oxford', 'MRC Laboratory of Molecular Biology', 'Yale University', 'Cornell University', 'Harvard Medical School', 'University College', 'University of Heidelberg', 'Berlin University', 'London University', 'Rockefeller Institute for Medical Research', 'Sorbonne University', 'Institut Pasteur', 'Goettingen University', 'National Institutes of Health', 'Bell Laboratories', 'University of Texas Southwestern Medical Center at Dallas', 'Uppsala University', 'Washington University', 'University of Pennsylvania', 'Eidgenössische Technische Hochschule (Swiss Federal Institute of Technology)', 'Karolins